<a href="https://colab.research.google.com/github/jaadu-1/Algo-trading/blob/main/BB84_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

"""
BB84 Quantum Key Distribution (QKD) – Educational Simulator
-----------------------------------------------------------
A compact, dependency‑free BB84 simulator you can run and modify.

Features:
- Random bit and basis generation for Alice and Bob
- Optional intercept‑resend eavesdropper (Eve) with configurable probability
- Basis sifting
- QBER estimation from a revealed sample
- Simple privacy amplification (hash‑and‑truncate)

Note: Real QKD also requires robust error reconciliation (e.g., Cascade/LDPC).
This simulator focuses on the core ideas for learning and experimentation.
"""

from dataclasses import dataclass
import random
import hashlib
from typing import List, Tuple

Basis = int  # 0 => rectilinear (+), 1 => diagonal (x)
Bit = int    # 0 or 1

@dataclass
class BB84Config:
    n_qubits: int = 256                   # number of transmitted qubits
    eve_intercept_prob: float = 0.0       # probability Eve intercepts a given qubit
    qber_check_fraction: float = 0.2      # fraction of sifted bits revealed to estimate QBER
    max_qber_threshold: float = 0.11      # abort if estimated QBER exceeds this
    final_key_bits: int = 64              # target bits after privacy amplification

@dataclass
class BB84Run:
    alice_bits: List[Bit]
    alice_bases: List[Basis]
    bob_bases: List[Basis]
    bob_results: List[Bit]
    sift_indices: List[int]
    sift_alice: List[Bit]
    sift_bob: List[Bit]
    revealed_indices: List[int]
    qber_estimate: float
    raw_key: List[Bit]
    final_key: str

def random_bits(n: int) -> List[Bit]:
    return [random.randint(0,1) for _ in range(n)]

def random_bases(n: int) -> List[Basis]:
    return [random.randint(0,1) for _ in range(n)]

def intercept_resend(bit: Bit, basis: Basis, eve_basis: Basis) -> Tuple[Bit, Basis]:
    """
    Eve measures using eve_basis, then resends a qubit encoded from her result
    in the *same* eve_basis (classic intercept-resend).
    If eve_basis != basis, her measurement is random -> introduces errors.
    Returns the (new_bit, new_basis) that Bob receives.
    """
    if eve_basis == basis:
        measured = bit  # perfect if basis matches
    else:
        measured = random.randint(0,1)  # random outcome if wrong basis
    # Eve resends encoded in her basis:
    return measured, eve_basis

def channel_with_eve(alice_bits: List[Bit],
                     alice_bases: List[Basis],
                     eve_intercept_prob: float) -> Tuple[List[Bit], List[Basis]]:
    """
    Simulate a quantum channel under intercept-resend attack.
    Returns the (transmitted_bits, transmitted_bases) that reach Bob.
    """
    n = len(alice_bits)
    out_bits = []
    out_bases = []
    for i in range(n):
        if random.random() < eve_intercept_prob:
            eve_basis = random.randint(0,1)
            new_bit, new_basis = intercept_resend(alice_bits[i], alice_bases[i], eve_basis)
            out_bits.append(new_bit)
            out_bases.append(new_basis)
        else:
            out_bits.append(alice_bits[i])
            out_bases.append(alice_bases[i])
    return out_bits, out_bases

def bob_measure(transmitted_bits: List[Bit],
                transmitted_bases: List[Basis],
                bob_bases: List[Basis]) -> List[Bit]:
    """
    Bob measures each qubit using his random basis.
    If bob_basis == transmitted_basis -> correct bit.
    Else -> random bit (measurement in incompatible basis).
    """
    n = len(transmitted_bits)
    results = []
    for i in range(n):
        if bob_bases[i] == transmitted_bases[i]:
            results.append(transmitted_bits[i])
        else:
            results.append(random.randint(0,1))
    return results

def sift(alice_bases: List[Basis], bob_bases: List[Basis],
         alice_bits: List[Bit], bob_results: List[Bit]) -> Tuple[List[int], List[Bit], List[Bit]]:
    """
    Keep positions where Alice and Bob used the same basis.
    """
    indices = [i for i,(a,b) in enumerate(zip(alice_bases, bob_bases)) if a == b]
    sift_alice = [alice_bits[i] for i in indices]
    sift_bob   = [bob_results[i] for i in indices]
    return indices, sift_alice, sift_bob

def estimate_qber(sift_alice: List[Bit], sift_bob: List[Bit], fraction: float) -> Tuple[List[int], float]:
    """
    Reveal a random subset of the sifted key to estimate QBER.
    Returns (revealed_indices, qber_estimate). The revealed bits should be discarded.
    """
    m = len(sift_alice)
    k = max(1, int(m * fraction)) if m > 0 else 0
    idx = random.sample(range(m), k) if m >= k and k > 0 else []
    if not idx:
        return [], 0.0
    errors = sum(1 for i in idx if sift_alice[i] != sift_bob[i])
    return idx, errors / len(idx)

def remove_indices(bits: List[Bit], indices_to_remove: List[int]) -> List[Bit]:
    remove_set = set(indices_to_remove)
    return [b for i,b in enumerate(bits) if i not in remove_set]

def bits_to_bytes(bits: List[Bit]) -> bytes:
    """
    Pack bit list into bytes, MSB-first per byte.
    """
    if not bits:
        return b""
    # pad to multiple of 8
    pad = (-len(bits)) % 8
    bits = bits + [0]*pad
    out = bytearray()
    for i in range(0, len(bits), 8):
        byte = 0
        for j in range(8):
            byte = (byte << 1) | bits[i+j]
        out.append(byte)
    return bytes(out)

def privacy_amplification(raw_key: List[Bit], final_bits: int) -> str:
    """
    One-shot hash-and-truncate using SHA-256; returns hex string.
    This is a simple stand-in for universal hashing.
    """
    data = bits_to_bytes(raw_key)
    digest = hashlib.sha256(data).digest()
    # If final_bits > 256, iterate the hash to stretch (not cryptographically robust),
    # but we only *truncate* here for simplicity.
    n_bytes = max(1, final_bits // 8)
    return digest[:n_bytes].hex()

def run_bb84(cfg: BB84Config) -> BB84Run:
    # 1) Alice prepares random bits and bases
    alice_bits  = random_bits(cfg.n_qubits)
    alice_bases = random_bases(cfg.n_qubits)
    # 2) Quantum channel (optional Eve intercept-resend)
    tx_bits, tx_bases = channel_with_eve(alice_bits, alice_bases, cfg.eve_intercept_prob)
    # 3) Bob chooses bases and measures
    bob_bases   = random_bases(cfg.n_qubits)
    bob_results = bob_measure(tx_bits, tx_bases, bob_bases)
    # 4) Sifting
    sift_indices, sift_alice, sift_bob = sift(alice_bases, bob_bases, alice_bits, bob_results)
    # 5) QBER estimation
    revealed_indices, qber_est = estimate_qber(sift_alice, sift_bob, cfg.qber_check_fraction)
    # Remove revealed indices from both parties' sifted keys
    sift_alice_kept = remove_indices(sift_alice, revealed_indices)
    sift_bob_kept   = remove_indices(sift_bob,   revealed_indices)
    # Compute raw key as Bob's kept bits (assuming low error). In a real protocol, error correction aligns them.
    raw_key = sift_bob_kept
    # Abort if QBER too high
    if qber_est > cfg.max_qber_threshold:
        final_key_hex = ""
    else:
        # 6) Privacy amplification
        final_key_hex = privacy_amplification(raw_key, cfg.final_key_bits)
    return BB84Run(
        alice_bits=alice_bits,
        alice_bases=alice_bases,
        bob_bases=bob_bases,
        bob_results=bob_results,
        sift_indices=sift_indices,
        sift_alice=sift_alice,
        sift_bob=sift_bob,
        revealed_indices=revealed_indices,
        qber_estimate=qber_est,
        raw_key=raw_key,
        final_key=final_key_hex
    )

# Convenience pretty-printers

def to_symbols(bits: List[int]) -> str:
    return "".join("0" if b == 0 else "1" for b in bits)

def bases_to_str(bases: List[int]) -> str:
    return "".join("+" if b == 0 else "x" for b in bases)

def demo(cfg: BB84Config) -> None:
    run = run_bb84(cfg)
    print("=== BB84 DEMO ===")
    print(f"n_qubits: {cfg.n_qubits}, Eve intercept prob: {cfg.eve_intercept_prob:.2f}")
    print(f"sifted size: {len(run.sift_indices)}")
    print(f"revealed for QBER: {len(run.revealed_indices)}")
    print(f"QBER estimate: {run.qber_estimate:.3%}")
    if run.final_key:
        print(f"raw key length (post-reveal): {len(run.raw_key)} bits")
        print(f"final key (SHA-256 truncated to {cfg.final_key_bits} bits): 0x{run.final_key}")
    else:
        print("ABORT: QBER too high; key discarded")

if __name__ == "__main__":
    random.seed(42)  # for reproducibility in the demo
    cfg = BB84Config(
        n_qubits=256,
        eve_intercept_prob=0.0,   # try 0.1 or 0.25 to see QBER rise
        qber_check_fraction=0.2,
        max_qber_threshold=0.11,
        final_key_bits=64
    )
    demo(cfg)


=== BB84 DEMO ===
n_qubits: 256, Eve intercept prob: 0.00
sifted size: 142
revealed for QBER: 28
QBER estimate: 0.000%
raw key length (post-reveal): 114 bits
final key (SHA-256 truncated to 64 bits): 0x05460d40bafd935b


In [2]:
# Paste this AFTER your simulator code

import gradio as gr

def run_demo(n_qubits, eve_prob, qber_fraction, max_qber, final_bits):
    cfg = BB84Config(
        n_qubits=int(n_qubits),
        eve_intercept_prob=float(eve_prob),
        qber_check_fraction=float(qber_fraction),
        max_qber_threshold=float(max_qber),
        final_key_bits=int(final_bits)
    )
    result = run_bb84(cfg)
    if result.final_key:
        return f"Sifted size: {len(result.sift_indices)}\nQBER estimate: {result.qber_estimate:.3%}\nRaw key length: {len(result.raw_key)} bits\nFinal key: 0x{result.final_key}"
    else:
        return f"ABORT: QBER too high ({result.qber_estimate:.3%}), key discarded."

with gr.Blocks() as demo:
    gr.Markdown("# BB84 Quantum Key Distribution Simulator")
    with gr.Row():
        n_qubits = gr.Number(value=256, label="Number of Qubits")
        eve_prob = gr.Slider(0,1,value=0,label="Eve Intercept Probability")
        qber_fraction = gr.Slider(0,1,value=0.2,label="QBER Check Fraction")
    with gr.Row():
        max_qber = gr.Number(value=0.11, label="Max QBER Threshold")
        final_bits = gr.Number(value=64, label="Final Key Bits")
    run_btn = gr.Button("Run BB84")
    output = gr.Textbox(label="Simulation Output")

    run_btn.click(run_demo, inputs=[n_qubits, eve_prob, qber_fraction, max_qber, final_bits], outputs=output)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9db16bd8ee3bfb9fc2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
